## Numerical simulation to obtain first-passage probability $H_n(s_0|s_1)$ and $F_n(s_0|s_1)$ on HCP lattice

In [1]:
import sys
import math
import numpy as np
from numpy import pi, sqrt, inf, log
from ecell4 import *

L = 100
voxel_radius = 0.005
D = 1
rng = GSLRandomNumberGenerator()
rng.seed(1)
dt = (4 * voxel_radius * voxel_radius) / (6 * D)
maxstep= 5
p=1
w = spatiocyte.create_spatiocyte_world_cell_list_impl(ones() * L, voxel_radius, Integer3(3, 3, 3), rng)

def singlerunA(dt, Pacc,alpha): #for evaluating G_n
    assert Pacc*alpha  <= 1
    assert alpha<=1
    coord1 = w.position2coordinate(ones() * L * 0.5)
    coord2 = w.get_neighbor(coord1, 0)
    dt = dt*alpha
    t, nsteps = 0.0, 0
    while all(0 <= dim < L for dim in w.coordinate2position(coord2)) and nsteps < maxstep:
        rnd = w.rng().uniform_int(0, 11)
        newcoord = w.get_neighbor(coord2, rnd)
        if newcoord == coord1:
            if Pacc * alpha >= w.rng().uniform(0, 1):
                return nsteps+1
        elif alpha >= w.rng().uniform(0, 1):
            coord2 = newcoord
            nsteps += 1
        t += dt     
    return -1

def singlerunB(dt, Pacc): #for evaluating H_n
    assert Pacc<= 1
    coord1 = w.position2coordinate(ones() * L * 0.5)
    coord2 = w.get_neighbor(coord1, 0)
    dt = dt
    t, nsteps = 0.0, 0
    while all(0 <= dim < L for dim in w.coordinate2position(coord2)) and nsteps < maxstep:
        rnd = w.rng().uniform_int(0, 11)    
        newcoord = w.get_neighbor(coord2, rnd)
        if newcoord == coord1:
            if Pacc >= w.rng().uniform(0, 1):
                return nsteps+1
        else:
            coord2 = newcoord
        t += dt
        nsteps += 1
    return -1

### insert the reaction acceptance probability and number of runs

In [2]:
Pacc = 2  #for evaluating G_n, (P_a=2a, a=1/2)
#Pacc=0.5 #for evaluating H_n (P_a=0.5)

#number of runs
ntrials = 100#int(1e9), used in the paper

if Pacc>1:
    res = [singlerunA(dt, Pacc,p/Pacc) for _ in range(ntrials)]
else:    
    res = [singlerunB(dt, Pacc) for _ in range(ntrials)]

#list of step taken to return to origin    
retstep = np.array([i for i in res if i!=-1]) 

### Comparison of numerical result with exact values

In [3]:
def err(a,b):
    return (a-b)*100/a

stp,stpc=np.unique(retstep,return_counts=True)
#exact first-passage probability for comparison
#P_a=0.5
p05=np.array([0.04166666,0.015625,0.01077836,0.0074297,0.005680163])
#P_a=2a, a=1/2
p2=np.array([0.153846,0.0473373,0.0313306,0.0200584,0.0147588]) 

for i,j in enumerate(stp):
    fp = stpc[i]/ntrials #numerical first-passage probability
    #print('n',j,'exact',p2[i],'numerical',fp,'error %',err(p2[i],fp))
    #print('n',j,'exact',p05[i],'numerical',fp,'error %',err(p05[i],fp))